In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import os
import cv2
from PIL import Image
import plotly.express as px
import plotly.graph_objects as go
from skimage.feature.texture import graycomatrix, graycoprops
from skimage.measure import label, regionprops
from skimage.measure import moments_hu
from tqdm import tqdm
from sklearn.cluster import DBSCAN
from scipy.stats import spearmanr
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from collections import defaultdict
import shutil
import hashlib

Définitions des fonctions statistiques et graphiques des caractéristiques

In [2]:
df = pd.read_csv("/workspaces/datasciencetest_reco_plante/notebooks/plant_V_Seg_all_features.csv")
df.head()

,ID_Image,nom_plante,nom_maladie,Est_Saine,Image_Path,dimensions,aire,périmètre,circularité,excentricité,...,mean_S,mean_V,sharpness_laplacian_var,hu_1,hu_2,hu_3,hu_4,hu_5,hu_6,hu_7
0,1,Tomato,Tomato_mosaic_virus,False,/workspaces/datasciencetest_reco_plante/datase...,126x184,12107.0,1025.151363,0.144767,0.702026,...,46.742599,19.742691,460.229553,2.674085,6.151797,9.510154,9.419995,-10.0,9.998989,-10.0
1,2,Tomato,Tomato_mosaic_virus,False,/workspaces/datasciencetest_reco_plante/datase...,162x175,15930.5,982.364566,0.207441,0.749598,...,51.395752,35.314026,696.851990,2.752474,5.969129,9.430474,9.807913,10.0,9.999964,10.0
2,3,Tomato,Tomato_mosaic_virus,False,/workspaces/datasciencetest_reco_plante/datase...,142x184,11486.5,854.482317,0.197693,0.909607,...,43.810669,24.449112,704.671448,2.568956,5.279443,9.058425,9.229536,10.0,9.995215,10.0
3,4,Tomato,Tomato_mosaic_virus,False,/workspaces/datasciencetest_reco_plante/datase...,176x198,18785.0,1044.749342,0.216270,0.659128,...,67.089340,48.410873,1098.690247,2.853978,6.610883,9.403455,9.948396,-10.0,-9.999976,-10.0
4,5,Tomato,Tomato_mosaic_virus,False,/workspaces/datasciencetest_reco_plante/datase...,148x188,15335.0,921.210238,0.227079,0.272318,...,54.958466,31.481064,481.223633,2.755678,6.210397,9.270586,9.841353,10.0,9.999856,10.0


In [3]:
# Analyse statistique - Violinplot interactif Plotly (distribution par classe)
def plot_violin_interactive(df, feature, classe='nom_plante'):
    fig = px.violin(df, y=feature, x=classe, box=True, points="all", hover_data=df.columns,
        title=f"Distribution de {feature} par {classe}")
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()

In [4]:
# Analyse statistique - Boxplot interactif Plotly
def plot_box_interactive(df, feature, classe='nom_plante'):
    fig = px.box(df, x=classe, y=feature, points="all", title=f"Boxplot de {feature} par {classe}",hover_data=df.columns)
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()

In [ ]:
# Graphiques recommandés pour Objectif 2 : Déterminer si la plante est malade
# Comparer l’état (Est_Saine) pour les features de couleur/texture

plot_box_interactive(df, feature='contrast', classe='Est_Saine')
plot_violin_interactive(df, feature='mean_R', classe='Est_Saine')
plot_box_interactive(df, feature='energy', classe='Est_Saine')